In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/Jeniejean/ANALYTICS-AND-TOOLS-AND-PROGRAMMING/main/Football_data.csv')
print(df)

    Div        Date   Time        HomeTeam        AwayTeam  FTHG  FTAG FTR  \
0    E0  13/08/2021  20:00       Brentford         Arsenal     2     0   H   
1    E0  14/08/2021  12:30      Man United           Leeds     5     1   H   
2    E0  14/08/2021  15:00         Burnley        Brighton     1     2   A   
3    E0  14/08/2021  15:00         Chelsea  Crystal Palace     3     0   H   
4    E0  14/08/2021  15:00         Everton     Southampton     3     1   H   
..   ..         ...    ...             ...             ...   ...   ...  ..   
375  E0  22/05/2022  16:00  Crystal Palace      Man United     1     0   H   
376  E0  22/05/2022  16:00       Leicester     Southampton     4     1   H   
377  E0  22/05/2022  16:00       Liverpool          Wolves     3     1   H   
378  E0  22/05/2022  16:00        Man City     Aston Villa     3     2   H   
379  E0  22/05/2022  16:00         Norwich       Tottenham     0     5   A   

     HTHG  HTAG  ... AvgC<2.5  AHCh  B365CAHH  B365CAHA  PCAHH 

**Simple Stats - 3 Questions:**

1. Who was the top judge in the competition for the year 2021-2022?
2. Which home team and which visiting team scored the most goals at the end of the competition?
3. Which team committed the most fouls, and how many did they commit? What were some of the fouls?

**Advanced - 2 Questions:**

4. Which website is the most worthwhile for sports betting?
5. Does the time period during the competition have an impact on goal scoring?

In [ ]:
#1
referee_counts = df['Referee'].value_counts().head(1)

print('top judge in the competition for the year 2021-2022 is:', referee_counts)

top judge in the competition for the year 2021-2022 is: A Taylor    28
Name: Referee, dtype: int64


In [ ]:
#2. ทีมเจ้าบ้าน และทีมเยือนใด ใดทำประตูได้เยอะที่สุดหลังจบการแข่งขัน
home_goals = df.groupby('HomeTeam')['FTHG'].sum().reset_index(name='Total Goals Home')
away_goals = df.groupby('AwayTeam')['FTAG'].sum().reset_index(name='Total Goals Away')

home_goals_sort = home_goals.sort_values(by='Total Goals Home', ascending=False)
away_goals_sort = away_goals.sort_values(by='Total Goals Away', ascending=False)

compare_goals = pd.merge(
    home_goals_sort,
    away_goals_sort,
    left_on='HomeTeam',
    right_on='AwayTeam')

print(compare_goals)

          HomeTeam  Total Goals Home        AwayTeam  Total Goals Away
0         Man City                58        Man City                41
1        Liverpool                49       Liverpool                45
2        Tottenham                38       Tottenham                31
3          Chelsea                37         Chelsea                39
4          Arsenal                35         Arsenal                26
5        Leicester                34       Leicester                28
6         West Ham                33        West Ham                27
7       Man United                32      Man United                25
8      Aston Villa                29     Aston Villa                23
9   Crystal Palace                27  Crystal Palace                23
10         Everton                27         Everton                16
11       Newcastle                26       Newcastle                18
12     Southampton                23     Southampton                20
13    

In [ ]:
#3. ทีมใดกระทำความผิดเยอะที่สุด และจำนวนกี่ใบ มีอะไรบ้าง
#หาTotal false ทั้งหมดก่อนนะ อันดับแรกเลย
home_f = df.groupby('HomeTeam')['HF'].sum().reset_index(name='Total HF')
away_f = df.groupby('AwayTeam')['AF'].sum().reset_index(name='Total AF')
ha_f = pd.merge(home_f, away_f, left_on='HomeTeam', right_on='AwayTeam')
ha_f['Total Falses'] = ha_f['Total HF'] + ha_f['Total AF']
ha_f_sorted = ha_f.sort_values('Total Falses', ascending=False)


#ทีนี้มาหาใบเหลืองของแต่ละทีมต่อ
h_yellow = df.groupby('HomeTeam')['HY'].sum().reset_index(name='Total HY')
a_yellow = df.groupby('AwayTeam')['AY'].sum().reset_index(name='Total AY')

ha_y = pd.merge(h_yellow, a_yellow, left_on='HomeTeam', right_on='AwayTeam')
ha_y['Total YCards'] = ha_y['Total HY'] + ha_y['Total AY']


#ทีนี้มาหาใบแดงของแต่ละทีมต่อ
h_red = df.groupby('HomeTeam')['HR'].sum().reset_index(name='Total HR')
a_red = df.groupby('AwayTeam')['AR'].sum().reset_index(name='Total AR')

ha_r = pd.merge(h_red, a_red, left_on='HomeTeam', right_on='AwayTeam')
ha_r['Total RCards'] = ha_r['Total HR'] + ha_r['Total AR']


#เอาทั้งสามอย่างที่หาก่อนหน้านี้มารวมกัน
ha_f_y = pd.merge(ha_f_sorted, ha_y, on='HomeTeam')
ha_f_y_r = pd.merge(ha_f_y, ha_r, on='HomeTeam')
ha_f_y_r['Total Cards'] = ha_f_y_r['Total YCards'] + ha_f_y_r['Total RCards']

#สั่งปริ้นเลย
print(ha_f_y_r[['HomeTeam', 'Total Falses', 'Total YCards', 'Total RCards', 'Total Cards']].head(10))

         HomeTeam  Total Falses  Total YCards  Total RCards  Total Cards
0           Leeds           469           100             3          103
1         Watford           440            57             3           60
2  Crystal Palace           415            67             1           68
3         Chelsea           409            63             1           64
4     Southampton           402            63             2           65
5     Aston Villa           401            77             2           79
6      Man United           395            75             2           77
7         Burnley           394            68             2           70
8        Brighton           393            71             2           73
9       Newcastle           391            79             1           80


In [ ]:
#4. ควรเล่นพนันกับเว็บไหนคุ้มที่สุด

roll = df['Div'].count()

h_avg = {
    'B365': df['B365H'].sum() / roll,
    'Bet&Win': df['BWH'].sum() / roll,
    'Interwetten': df['IWH'].sum() / roll,
    'Pinnacle Sports': df['PSH'].sum() / roll,
    'William Hill': df['WHH'].sum() / roll,
    'BetVictor': df['VCH'].sum() / roll
}

d_avg = {
    'B365': df['B365D'].sum() / roll,
    'Bet&Win': df['BWD'].sum() / roll,
    'Interwetten': df['IWD'].sum() / roll,
    'Pinnacle Sports': df['PSD'].sum() / roll,
    'William Hill': df['WHD'].sum() / roll,
    'BetVictor': df['VCD'].sum() / roll
}

a_avg = {
    'B365': df['B365A'].sum() / roll,
    'Bet&Win': df['BWA'].sum() / roll,
    'Interwetten': df['IWA'].sum() / roll,
    'Pinnacle Sports': df['PSA'].sum() / roll,
    'William Hill': df['WHA'].sum() / roll,
    'BetVictor': df['VCA'].sum() / roll
}

best_h_company, best_h_rate = max(h_avg.items(), key=lambda item: item[1])
best_d_company, best_d_rate = max(d_avg.items(), key=lambda item: item[1])
best_a_company, best_a_rate = max(a_avg.items(), key=lambda item: item[1])

best_company = max(best_h_company, best_d_company, best_a_company)

# Print out the best rates with two decimal places and company names
print(f"ราคาเดิมพันเฉลี่ยสำหรับการชนะของทีมเจ้าบ้านสูงที่สุดคือ: {best_h_rate:.2f} จาก {best_h_company}")
print(f"ราคาเดิมพันเฉลี่ยสำหรับการเสมอสูงที่สุดคือ: {best_d_rate:.2f} จาก {best_d_company}")
print(f"ราคาเดิมพันเฉลี่ยสำหรับการชนะของทีมเยือนสูงที่สุดคือ: {best_a_rate:.2f} จาก {best_a_company}")

print('ดังนั้นควรเล่นพนันกับเว็บไซต์', best_company)

ราคาเดิมพันเฉลี่ยสำหรับการชนะของทีมเจ้าบ้านสูงที่สุดคือ: 3.17 จาก Pinnacle Sports
ราคาเดิมพันเฉลี่ยสำหรับการเสมอสูงที่สุดคือ: 4.34 จาก Pinnacle Sports
ราคาเดิมพันเฉลี่ยสำหรับการชนะของทีมเยือนสูงที่สุดคือ: 5.01 จาก Pinnacle Sports
ดังนั้นควรเล่นพนันกับเว็บไซต์ Pinnacle Sports


In [ ]:
#5. ช่วงเวลาในการแข่งขัน มีผลต่อการทำประตูหรือไม่

time_count = df['Time'].value_counts().reset_index().rename(columns={'index':'Time', 'Time':'Time Count'})

t_h = df.groupby('Time')['FTHG'].sum().reset_index().rename(columns={'FTHG':'Home Goals'})
t_a = df.groupby('Time')['FTAG'].sum().reset_index().rename(columns={'FTAG':'Away Goals'})

total_goals = pd.merge(t_h, t_a, on='Time')
total_goals['Total Goals'] = total_goals['Home Goals'] + total_goals['Away Goals']

time_c = pd.merge(time_count, total_goals, on='Time')

#เอาประตูที่ทำได้หารเวลาเริ่มเกม
time_c['Winning Avg'] = time_c['Total Goals'] / time_c['Time Count']
time_c['Winning Avg'] = time_c['Winning Avg'].round(3)

b_t = time_c[time_c['Winning Avg'] == time_c['Winning Avg'].max()]
w_t = time_c[time_c['Winning Avg'] == time_c['Winning Avg'].min()]

best = b_t['Time'].iloc[0]
best_avg = b_t['Winning Avg'].iloc[0]
worst = w_t['Time'].iloc[0]
worst_avg = w_t['Winning Avg'].iloc[0]


print(time_c[['Time', 'Time Count', 'Total Goals', 'Winning Avg']])


print('เวลาเริ่มเกมที่สามารถทำประตูได้ดีที่สุดคือเวลา:', best, 'ด้วยอัตราการทำประตูที่:', best_avg)
print('เวลาเริ่มเกมที่สามารถทำประตูได้แย่ที่สุดคือเวลา:', worst, 'ด้วยอัตราการทำประตูที่:', worst_avg)

     Time  Time Count  Total Goals  Winning Avg
0   15:00         118          326        2.763
1   14:00          66          172        2.606
2   20:00          37          105        2.838
3   17:30          31           90        2.903
4   12:30          30           75        2.500
5   16:30          29          103        3.552
6   19:45          27           72        2.667
7   19:30          18           52        2.889
8   16:00          10           39        3.900
9   20:15           9           29        3.222
10  14:15           2            5        2.500
11  18:00           1            0        0.000
12  15:30           1            2        2.000
13  12:00           1            1        1.000
เวลาเริ่มเกมที่สามารถทำประตูได้ดีที่สุดคือเวลา: 16:00 ด้วยอัตราการทำประตูที่: 3.9
เวลาเริ่มเกมที่สามารถทำประตูได้แย่ที่สุดคือเวลา: 18:00 ด้วยอัตราการทำประตูที่: 0.0
